# 多様性サンプリング (Diversity Sampling) チュートリアル

## 概要

能動学習のサンプリング戦略として、`active_learning_demo.ipynb` で扱った **不確実性サンプリング** と対比しながら、**多様性サンプリング** を学びます。

| 戦略 | 判断基準 | 強み | 弱み |
|------|----------|------|------|
| **不確実性サンプリング** | モデルの予測標準偏差が最大の点を選ぶ | モデルの弱点を素早く補う | 高不確実領域にクラスタリングしやすい |
| **多様性サンプリング** | 既存データから最も遠い点を選ぶ | 空間を均等にカバーする | モデルの現在の状態を考慮しない |
| **ハイブリッド** | 不確実性×多様性の重み付き和 | 両者の利点を組み合わせる | ハイパーパラメータ α の調整が必要 |

### このノートブックで学ぶこと
1. **1D 例**：不確実性サンプリングのクラスタリング問題を可視化
2. **多様性スコア**：Greedy K-Center（最大最小距離）の仕組み
3. **ハイブリッド獲得関数**：α で不確実性と多様性を連続的にブレンド
4. **2D 定量比較**：ガウス過程回帰 (GPR) を基準モデルとして RMSE で比較

## 1. 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as _fm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import cdist
from pathlib import Path

# ─── 日本語フォント設定 ──────────────────────────────────────────────────────
_jp_candidates = ["Yu Gothic", "Meiryo", "BIZ UDGothic", "MS Gothic"]
_available = {f.name for f in _fm.fontManager.ttflist}
_jp_font = next((f for f in _jp_candidates if f in _available), None)
if _jp_font:
    matplotlib.rcParams["font.family"] = _jp_font
    print(f"[font] Japanese font set: {_jp_font}")
else:
    print("[font] No Japanese font found — labels will be shown in English")
matplotlib.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.dpi"] = 120
plt.rcParams["font.size"] = 10

DATA_DIR = Path("data")

In [ ]:
# ─── GPR ヘルパー ─────────────────────────────────────────────────────────────
def build_gpr(X_train, y_train):
    """ガウス過程回帰モデルを構築・学習する"""
    kernel = ConstantKernel(1.0) * RBF(length_scale=0.3) + WhiteKernel(noise_level=0.01)
    gpr = GaussianProcessRegressor(
        kernel=kernel, n_restarts_optimizer=5, random_state=42
    )
    gpr.fit(X_train, y_train)
    return gpr


def evaluate_model(gpr, X_eval, y_eval):
    """グリッド全体でのRMSEを計算"""
    return np.sqrt(mean_squared_error(y_eval, gpr.predict(X_eval)))


# ─── スコア正規化 ─────────────────────────────────────────────────────────────
def normalize(v):
    """0-1 正規化（最小値=0, 最大値=1）"""
    vmin, vmax = v.min(), v.max()
    return (v - vmin) / (vmax - vmin + 1e-10)

In [ ]:
# ─── 2D データ読み込み（active_learning_demo.ipynb と共通）──────────────────
grid    = np.load(DATA_DIR / "grid.npz")
initial = np.load(DATA_DIR / "initial_samples.npz")
pool    = np.load(DATA_DIR / "candidate_pool.npz")

X1_grid, X2_grid, Y_grid = grid["X1"], grid["X2"], grid["Y"]
X_grid_flat = np.column_stack([X1_grid.ravel(), X2_grid.ravel()])
y_grid_flat = Y_grid.ravel()

print(f"初期サンプル: {initial['X'].shape[0]} 点")
print(f"候補プール:   {pool['X'].shape[0]} 点")
print(f"評価グリッド: {X1_grid.shape[0]}×{X1_grid.shape[1]} = {y_grid_flat.shape[0]} 点")

## 2. 不確実性サンプリングのクラスタリング問題（1D 例）

### なぜ問題が起きるのか

不確実性サンプリングは「モデルが最も自信のない点」を選びます。  
しかし、1点サンプリングするたびにその近傍の不確実性も下がるため、**同じ領域に連続してサンプルが集中しやすい**という性質があります。

```
初期状態:  左端の不確実性 ≈ 右端の不確実性（どちらも高い）
1点目:     左端の x=0.02 を選択（わずかに左が高かった）
2点目:     x=0.02 の隣 x=0.05 がまだ不確実 → また左端
3点目:     x=0.08 がまだ不確実 → また左端
    ↓
右端はずっと未探索のまま！
```

多様性サンプリングはこの問題を「既存データとの距離」で解決します。

In [ ]:
# ─── 1D デモ：クラスタリング問題の可視化 ─────────────────────────────────────

# 真の関数（非対称で左右の端が重要）
true_f_1d = lambda x: 1.5 * np.sin(3 * np.pi * x) + 0.3 * np.cos(7 * np.pi * x)

# 評価用グリッド
x_grid_1d = np.linspace(0, 1, 500).reshape(-1, 1)
y_grid_1d = true_f_1d(x_grid_1d.ravel())

# 初期学習データ（中央付近に6点）
np.random.seed(0)
X_init_1d = np.array([[0.28], [0.36], [0.44], [0.52], [0.60], [0.68]])
y_init_1d = true_f_1d(X_init_1d.ravel()) + np.random.normal(0, 0.05, len(X_init_1d))

# 候補プール：左端 [0.01, 0.24] と右端 [0.76, 0.99] に30点ずつ
X_pool_left  = np.linspace(0.01, 0.24, 24).reshape(-1, 1)
X_pool_right = np.linspace(0.76, 0.99, 24).reshape(-1, 1)
X_pool_1d = np.vstack([X_pool_left, X_pool_right])
y_pool_1d = true_f_1d(X_pool_1d.ravel()) + np.random.normal(0, 0.05, len(X_pool_1d))

print(f"初期学習データ: {len(X_init_1d)} 点（中央付近）")
print(f"候補プール: {len(X_pool_1d)} 点（左端 {len(X_pool_left)} + 右端 {len(X_pool_right)}）")

In [ ]:
N_STEPS_1D = 8  # 比較するステップ数


def run_1d_loop(strategy, n_steps=N_STEPS_1D):
    """1D デモ用の能動学習ループ（strategy: 'uncertainty' or 'diversity'）"""
    X_tr = X_init_1d.copy()
    y_tr = y_init_1d.copy()
    X_rem = X_pool_1d.copy()
    y_rem = y_pool_1d.copy()
    snapshots = []  # (X_tr, X_selected) のリスト

    for step in range(n_steps):
        gpr = build_gpr(X_tr, y_tr)

        if strategy == "uncertainty":
            _, std = gpr.predict(X_rem, return_std=True)
            scores = std
        else:  # diversity (greedy k-center)
            dists = cdist(X_rem, X_tr, metric="euclidean")
            scores = dists.min(axis=1)

        idx = int(np.argmax(scores))
        snapshots.append({
            "gpr": gpr,
            "X_train": X_tr.copy(),
            "new_point": X_rem[[idx]].copy(),
            "scores": scores.copy(),
            "X_pool_rem": X_rem.copy(),
        })

        X_tr  = np.vstack([X_tr, X_rem[[idx]]])
        y_tr  = np.concatenate([y_tr, y_rem[[idx]]])
        X_rem = np.delete(X_rem, idx, axis=0)
        y_rem = np.delete(y_rem, idx)

    return snapshots, X_tr


snaps_unc, X_final_unc = run_1d_loop("uncertainty")
snaps_div, X_final_div = run_1d_loop("diversity")

# ─── 可視化：3ステップごとの比較（ステップ 0, 2, 4, 6）─────────────────────
show_steps = [0, 2, 4, 6]
fig, axes = plt.subplots(2, len(show_steps), figsize=(16, 8), sharey=False)
fig.suptitle("1D デモ：不確実性サンプリング vs 多様性サンプリング\n"
             "黒●=初期データ, 赤★=今回選択点, 灰○=候補プール",
             fontsize=12, fontweight="bold")

strategy_labels = ["不確実性サンプリング (GPR std)", "多様性サンプリング (Greedy K-Center)"]
score_labels    = ["GPR 予測標準偏差 (std)",          "最近傍距離 (diversity score)"]
score_colors    = ["orangered",                         "steelblue"]

for row, snaps in enumerate([snaps_unc, snaps_div]):
    for col, step_idx in enumerate(show_steps):
        ax = axes[row, col]
        snap = snaps[step_idx]
        gpr  = snap["gpr"]

        # 真の関数
        ax.plot(x_grid_1d, y_grid_1d, "k--", lw=1, alpha=0.4, label="True f")

        # GPR 予測
        y_pred, y_std = gpr.predict(x_grid_1d, return_std=True)
        ax.plot(x_grid_1d, y_pred, color="tab:blue", lw=1.5, label="GPR pred")
        ax.fill_between(
            x_grid_1d.ravel(),
            y_pred - 2 * y_std, y_pred + 2 * y_std,
            alpha=0.15, color="tab:blue"
        )

        # 候補プール（小さく）
        ax.scatter(snap["X_pool_rem"], true_f_1d(snap["X_pool_rem"].ravel()),
                   c="silver", s=20, zorder=2)

        # 既存学習データ
        ax.scatter(snap["X_train"][:, 0], true_f_1d(snap["X_train"].ravel()),
                   c="k", s=40, zorder=4, label="Training data")

        # 今回選んだ点
        ax.scatter(snap["new_point"][:, 0], true_f_1d(snap["new_point"].ravel()),
                   c="red", marker="*", s=180, zorder=5,
                   label=f"Selected (step {step_idx+1})")

        # スコアを右y軸に
        ax2 = ax.twinx()
        ax2.bar(
            snap["X_pool_rem"].ravel(),
            snap["scores"],
            width=0.008, color=score_colors[row], alpha=0.4
        )
        ax2.set_ylabel(score_labels[row], fontsize=7, color=score_colors[row])
        ax2.tick_params(axis="y", colors=score_colors[row], labelsize=7)

        ax.set_title(f"Step {step_idx+1}", fontsize=9)
        ax.set_xlabel("x")
        ax.set_xlim(-0.02, 1.02)
        if col == 0:
            ax.set_ylabel(strategy_labels[row], fontsize=9)
        if row == 0 and col == 0:
            ax.legend(fontsize=7, loc="upper center")

plt.tight_layout()
plt.show()

In [ ]:
# ─── 選択された点の分布を比較 ─────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 初期 GPR 状態
gpr0 = build_gpr(X_init_1d, y_init_1d)
y0_pred, y0_std = gpr0.predict(x_grid_1d, return_std=True)

for ax in axes:
    ax.plot(x_grid_1d, y_grid_1d, "k--", lw=1.5, alpha=0.5, label="True function")

# Panel 1: 初期状態
ax = axes[0]
ax.plot(x_grid_1d, y0_pred, "tab:blue", lw=1.5, label="GPR prediction")
ax.fill_between(x_grid_1d.ravel(), y0_pred - 2*y0_std, y0_pred + 2*y0_std,
                alpha=0.2, color="tab:blue")
ax.scatter(X_init_1d, true_f_1d(X_init_1d.ravel()),
           c="k", s=60, zorder=5, label="Initial (6 pts)")
ax.scatter(X_pool_1d, true_f_1d(X_pool_1d.ravel()),
           c="silver", s=20, zorder=2, label="Pool")
ax.set_title("初期状態（中央に6点）")
ax.legend(fontsize=8)
ax.set_xlabel("x")

# Panel 2: 不確実性サンプリング 8ステップ後
ax = axes[1]
gpr_unc = build_gpr(X_final_unc, true_f_1d(X_final_unc.ravel()))
y_unc, y_unc_std = gpr_unc.predict(x_grid_1d, return_std=True)
ax.plot(x_grid_1d, y_unc, "tomato", lw=1.5, label="GPR prediction")
ax.fill_between(x_grid_1d.ravel(), y_unc - 2*y_unc_std, y_unc + 2*y_unc_std,
                alpha=0.2, color="tomato")
ax.scatter(X_init_1d, true_f_1d(X_init_1d.ravel()),
           c="k", s=40, zorder=4, label="Initial")
new_unc = X_final_unc[len(X_init_1d):]
ax.scatter(new_unc, true_f_1d(new_unc.ravel()),
           c="red", marker="^", s=80, zorder=5, label=f"Selected ({len(new_unc)} pts)")
n_left  = int((new_unc[:, 0] < 0.5).sum())
n_right = int((new_unc[:, 0] >= 0.5).sum())
ax.set_title(f"不確実性サンプリング ({N_STEPS_1D} steps)\n左端 {n_left}点, 右端 {n_right}点")
ax.legend(fontsize=8)
ax.set_xlabel("x")

# Panel 3: 多様性サンプリング 8ステップ後
ax = axes[2]
gpr_div = build_gpr(X_final_div, true_f_1d(X_final_div.ravel()))
y_div, y_div_std = gpr_div.predict(x_grid_1d, return_std=True)
ax.plot(x_grid_1d, y_div, "steelblue", lw=1.5, label="GPR prediction")
ax.fill_between(x_grid_1d.ravel(), y_div - 2*y_div_std, y_div + 2*y_div_std,
                alpha=0.2, color="steelblue")
ax.scatter(X_init_1d, true_f_1d(X_init_1d.ravel()),
           c="k", s=40, zorder=4, label="Initial")
new_div = X_final_div[len(X_init_1d):]
ax.scatter(new_div, true_f_1d(new_div.ravel()),
           c="steelblue", marker="^", s=80, zorder=5, label=f"Selected ({len(new_div)} pts)")
n_left  = int((new_div[:, 0] < 0.5).sum())
n_right = int((new_div[:, 0] >= 0.5).sum())
ax.set_title(f"多様性サンプリング ({N_STEPS_1D} steps)\n左端 {n_left}点, 右端 {n_right}点")
ax.legend(fontsize=8)
ax.set_xlabel("x")

for ax in axes:
    ax.set_xlim(-0.02, 1.02)
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("【観察ポイント】")
print(f"  不確実性サンプリング: 左端 {n_left}点 / 右端 {int((X_final_unc[len(X_init_1d):,0] >= 0.5).sum())}点")
print(f"  多様性サンプリング:   左端 {int((X_final_div[len(X_init_1d):,0] < 0.5).sum())}点 / 右端 {int((X_final_div[len(X_init_1d):,0] >= 0.5).sum())}点")

## 3. 多様性サンプリングの実装

### 3.1 Greedy K-Center（最大最小距離法）

最もシンプルな多様性サンプリング手法です。

$$
\text{diversity score}(x) = \min_{x' \in X_{\text{train}}} \| x - x' \|_2
$$

```
次のサンプル = argmax_{x ∈ 候補プール} min_{x' ∈ 学習データ} ||x - x'||₂
```

**直感**: 「学習データからできるだけ離れた点」を選ぶ = 空間を均等にカバーする

### 3.2 ガウス過程回帰との違い

| | GPR 不確実性サンプリング | Greedy K-Center |
|--|--|--|
| **情報源** | GPR が計算する予測標準偏差 | 学習データとの幾何学的距離 |
| **GPR の必要性** | 必要（毎ステップ再学習） | **不要**（距離計算のみ） |
| **感度** | 関数の局所的な複雑さに敏感 | 関数の形状を一切無視 |
| **クラスタリング** | 起きやすい | 起きにくい（設計上）|

In [ ]:
# ─── 多様性サンプリング関数 ──────────────────────────────────────────────────

def diversity_scores(X_pool, X_train):
    """
    Greedy K-Center の多様性スコア
    各候補点から学習データへの最小距離を返す
    """
    dists = cdist(X_pool, X_train, metric="euclidean")  # (N_pool, N_train)
    return dists.min(axis=1)  # 各候補点の最近傍距離


def uncertainty_scores(gpr, X_pool):
    """GPR の予測標準偏差（不確実性スコア）"""
    _, std = gpr.predict(X_pool, return_std=True)
    return std


def hybrid_scores(gpr, X_pool, X_train, alpha=0.5):
    """
    ハイブリッドスコア：α * 不確実性 + (1-α) * 多様性

    Parameters
    ----------
    alpha : float
        alpha=1.0 → 純粋な不確実性サンプリング
        alpha=0.0 → 純粋な多様性サンプリング
        alpha=0.5 → バランス（推奨デフォルト）
    """
    unc = uncertainty_scores(gpr, X_pool)
    div = diversity_scores(X_pool, X_train)
    return alpha * normalize(unc) + (1 - alpha) * normalize(div)

### 3.3 2D スコアマップの比較

初期データ 20 点で学習した GPR を使って、候補プール全体での各スコアを可視化します。

In [ ]:
# 初期 GPR 学習
gpr_init = build_gpr(initial["X"], initial["y"])

# グリッド全体でスコアを計算
unc_map  = uncertainty_scores(gpr_init, X_grid_flat).reshape(X1_grid.shape)
div_map  = diversity_scores(X_grid_flat, initial["X"]).reshape(X1_grid.shape)
hyb_map  = hybrid_scores(gpr_init, X_grid_flat, initial["X"], alpha=0.5).reshape(X1_grid.shape)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

titles   = ["GPR 不確実性スコア (std)",
            "多様性スコア（最近傍距離）",
            "ハイブリッドスコア (α=0.5)"]
maps     = [unc_map, div_map, hyb_map]
cmaps    = ["YlOrRd", "Blues", "Greens"]

for ax, score_map, title, cmap in zip(axes, maps, titles, cmaps):
    c = ax.contourf(X1_grid, X2_grid, score_map, levels=25, cmap=cmap)
    fig.colorbar(c, ax=ax)
    ax.scatter(initial["X"][:, 0], initial["X"][:, 1],
               c="k", s=30, edgecolors="w", linewidths=0.5, zorder=5,
               label="Training data")
    ax.set_title(title, fontsize=10, fontweight="bold")
    ax.set_xlabel("x1"); ax.set_ylabel("x2")
    ax.legend(fontsize=8, loc="upper right")

fig.suptitle("初期モデル (20点) における各スコアマップ",
             fontsize=12, fontweight="bold")
plt.tight_layout()
plt.show()

print("\n【スコアマップの読み方】")
print("  不確実性スコア: 学習データから遠い + 関数が複雑な領域が高い")
print("  多様性スコア:   純粋に学習データから遠い領域が高い（関数形状を無視）")
print("  ハイブリッド:   両者の平均（α=0.5）")

## 4. ハイブリッドアプローチ：α でバランスを制御

$$
\text{hybrid score}(x) = \underbrace{\alpha \cdot \hat{\sigma}_{\text{GPR}}(x)}_{\text{不確実性}} + \underbrace{(1-\alpha) \cdot \hat{d}(x)}_{\text{多様性}}
$$

ここで $\hat{\sigma}$, $\hat{d}$ はそれぞれ $[0, 1]$ に正規化した不確実性・多様性スコアです。

| α 値 | 戦略 | 適した場面 |
|------|------|----------|
| 1.0 | 純粋な不確実性 | モデルの弱点が局所的・関数が複雑 |
| 0.7 | 不確実性重視 | 不確実性情報を優先しつつ分散も考慮 |
| 0.5 | バランス | 一般的な推奨値 |
| 0.3 | 多様性重視 | 初期探索フェーズ・関数形状が不明 |
| 0.0 | 純粋な多様性 | 均等カバレッジを最優先 |

In [ ]:
# ─── α 値によるスコアマップの変化 ────────────────────────────────────────────
alphas = [1.0, 0.7, 0.5, 0.3, 0.0]

fig, axes = plt.subplots(1, len(alphas), figsize=(18, 4.5))
fig.suptitle("ハイブリッドスコアマップ：α を変えたとき", fontsize=12, fontweight="bold")

for ax, alpha in zip(axes, alphas):
    score = hybrid_scores(gpr_init, X_grid_flat, initial["X"], alpha=alpha)
    score_map = score.reshape(X1_grid.shape)

    c = ax.contourf(X1_grid, X2_grid, score_map, levels=25, cmap="plasma")
    fig.colorbar(c, ax=ax, shrink=0.8)

    # 最高スコアの点を★でマーク
    best_flat = int(np.argmax(score))
    best_x1, best_x2 = X_grid_flat[best_flat]
    ax.scatter(initial["X"][:, 0], initial["X"][:, 1],
               c="k", s=15, zorder=4)
    ax.scatter([best_x1], [best_x2], c="yellow", marker="*", s=250, zorder=6,
               edgecolors="k", linewidths=0.5, label="Best point")

    if alpha == 1.0:
        label = f"α={alpha:.1f}\n（純粋な不確実性）"
    elif alpha == 0.0:
        label = f"α={alpha:.1f}\n（純粋な多様性）"
    else:
        label = f"α={alpha:.1f}"
    ax.set_title(label, fontsize=9)
    ax.set_xlabel("x1")
    ax.legend(fontsize=7, loc="upper right")

axes[0].set_ylabel("x2")
plt.tight_layout()
plt.show()

## 5. 定量比較：GPR を基準モデルとした RMSE 評価

4 つの戦略を 30 ステップ実行して RMSE 学習曲線を比較します。  
いずれの戦略でも**最終的なモデルは GPR**（統一の評価モデル）を使用します。

```
不確実性: GPR 必須（毎ステップ再学習）
多様性:   GPR 不要（距離計算のみ）だが、RMSE 評価には GPR を使用
ハイブリッド: 両方使用
ランダム: GPR 不要（RMSE 評価のみ）
```

In [ ]:
def run_active_learning_2d(
    strategy,
    alpha=0.5,
    n_iterations=30,
    random_state=42,
):
    """
    2D データセットで能動学習を実行

    Parameters
    ----------
    strategy : 'uncertainty' | 'diversity' | 'hybrid' | 'random'
    alpha    : ハイブリッドの重み（alpha=1→不確実性, alpha=0→多様性）

    Returns
    -------
    dict with rmse_history, selected_points, X_train_final, gpr_final
    """
    rng = np.random.default_rng(random_state)
    X_tr   = initial["X"].copy()
    y_tr   = initial["y"].copy()
    X_rem  = pool["X"].copy()
    y_rem  = pool["y"].copy()

    rmse_history     = []
    selected_points  = []

    for _ in range(n_iterations):
        # 評価用 GPR（全戦略で共通）
        gpr = build_gpr(X_tr, y_tr)
        rmse_history.append(evaluate_model(gpr, X_grid_flat, y_grid_flat))

        # ─── スコア計算 ───────────────────────────────────────────────────
        if strategy == "uncertainty":
            scores = uncertainty_scores(gpr, X_rem)

        elif strategy == "diversity":
            scores = diversity_scores(X_rem, X_tr)

        elif strategy == "hybrid":
            scores = hybrid_scores(gpr, X_rem, X_tr, alpha=alpha)

        elif strategy == "random":
            scores = rng.random(len(X_rem))

        # ─── 最高スコアの点を選択 ─────────────────────────────────────────
        idx = int(np.argmax(scores))
        selected_points.append(X_rem[[idx]].copy())
        X_tr  = np.vstack([X_tr, X_rem[[idx]]])
        y_tr  = np.concatenate([y_tr, y_rem[[idx]]])
        X_rem = np.delete(X_rem, idx, axis=0)
        y_rem = np.delete(y_rem, idx)

    # 最終モデル
    gpr_final = build_gpr(X_tr, y_tr)
    rmse_history.append(evaluate_model(gpr_final, X_grid_flat, y_grid_flat))

    return {
        "rmse_history":    rmse_history,
        "selected_points": np.vstack(selected_points),
        "X_train_final":   X_tr,
        "gpr_final":       gpr_final,
    }


N_ITER = 30
print(f"4戦略を各 {N_ITER} ステップ実行します...\n")

strategies = [
    ("uncertainty", 0.5,  "Uncertainty (GPR std)"),
    ("diversity",   0.5,  "Diversity (K-Center)"),
    ("hybrid",      0.7,  "Hybrid α=0.7 (uncertainty-heavy)"),
    ("hybrid",      0.3,  "Hybrid α=0.3 (diversity-heavy)"),
    ("random",      0.5,  "Random"),
]

results = {}
for strat, alpha, label in strategies:
    print(f"  [{label}] 実行中...", end=" ", flush=True)
    key = label
    results[key] = run_active_learning_2d(strategy=strat, alpha=alpha, n_iterations=N_ITER)
    h = results[key]["rmse_history"]
    print(f"RMSE: {h[0]:.4f} → {h[-1]:.4f}")

print("\n完了！")

In [ ]:
# ─── 学習曲線の比較 ──────────────────────────────────────────────────────────
n_samples = np.arange(initial["X"].shape[0], initial["X"].shape[0] + N_ITER + 1)

style_map = {
    "Uncertainty (GPR std)":         ("tomato",      "o-",  2.0),
    "Diversity (K-Center)":          ("steelblue",   "s-",  2.0),
    "Hybrid α=0.7 (uncertainty-heavy)": ("darkorange",  "^-",  1.5),
    "Hybrid α=0.3 (diversity-heavy)": ("mediumorchid", "D-",  1.5),
    "Random":                        ("gray",         "x--", 1.2),
}

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# ─ 左：学習曲線 ──────────────────────────────────────────────────────────────
ax = axes[0]
for label, (color, marker, lw) in style_map.items():
    res = results[label]
    ms = 6 if "x" in marker else 4
    ax.plot(n_samples, res["rmse_history"], marker,
            label=label, color=color, markersize=ms, linewidth=lw, alpha=0.85)

ax.set_xlabel("Training Samples（学習データ数）")
ax.set_ylabel("RMSE")
ax.set_title("学習曲線：不確実性 vs 多様性 vs ハイブリッド")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# ─ 右：最終 RMSE 棒グラフ ─────────────────────────────────────────────────────
ax = axes[1]
labels  = list(style_map.keys())
finals  = [results[l]["rmse_history"][-1] for l in labels]
colors  = [style_map[l][0] for l in labels]

rand_final = results["Random"]["rmse_history"][-1]
bars = ax.bar(range(len(labels)), finals, color=colors, alpha=0.75,
              edgecolor="k", linewidth=0.8)
ax.axhline(rand_final, color="gray", linestyle="--", alpha=0.6, label="Random baseline")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels([l.replace(" (", "\n(") for l in labels], fontsize=8)
ax.set_ylabel("Final RMSE")
ax.set_title(f"最終 RMSE 比較 (n={n_samples[-1]})")
ax.grid(True, alpha=0.3, axis="y")
ax.legend(fontsize=9)

for bar, val in zip(bars, finals):
    imp = (1 - val / rand_final) * 100
    ax.text(bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.001,
            f"{val:.4f}\n({imp:+.1f}%)",
            ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.show()

print(f"\n{'':=<65}")
print(f"{'戦略':<35} {'最終RMSE':>10}  {'ランダム比改善率':>12}")
print(f"{'':=<65}")
for label, (color, marker, lw) in style_map.items():
    val = results[label]["rmse_history"][-1]
    imp = (1 - val / rand_final) * 100
    best_mark = " ★ Best" if val == min(results[l]["rmse_history"][-1] for l in results) else ""
    print(f"  {label:<33}: {val:>10.4f}  {imp:>+11.1f}%{best_mark}")

## 6. 選択点の空間分布：GPR の視点で見る違い

各戦略が「どこを選んだか」を可視化することで、戦略の特性をより深く理解します。

In [ ]:
# ─── 選択点の空間分布比較（主要3戦略） ──────────────────────────────────────
compare_keys = [
    "Uncertainty (GPR std)",
    "Diversity (K-Center)",
    "Hybrid α=0.5 (balanced)",  # ← 後で追加
]

# α=0.5 のハイブリッドを別途計算
if "Hybrid α=0.5 (balanced)" not in results:
    print("Hybrid α=0.5 を追加計算中...", end=" ", flush=True)
    results["Hybrid α=0.5 (balanced)"] = run_active_learning_2d(
        strategy="hybrid", alpha=0.5, n_iterations=N_ITER
    )
    print("完了")

fig, axes = plt.subplots(2, 3, figsize=(16, 11))

for col, key in enumerate(compare_keys):
    res  = results[key]
    gpr  = res["gpr_final"]
    X_tr = res["X_train_final"]
    sel  = res["selected_points"]  # 追加された点

    y_pred, y_std = gpr.predict(X_grid_flat, return_std=True)
    Y_pred = y_pred.reshape(X1_grid.shape)
    Y_std  = y_std.reshape(X1_grid.shape)
    Y_err  = np.abs(Y_pred - Y_grid)

    # ─ 上段：選択点の順序（色が選択順） ──────────────────────────────────────
    ax = axes[0, col]
    ax.contourf(X1_grid, X2_grid, Y_grid, levels=20, cmap="RdBu_r", alpha=0.25)
    ax.scatter(initial["X"][:, 0], initial["X"][:, 1],
               c="gray", edgecolors="k", s=40, zorder=3, label=f"Initial ({len(initial['X'])}pts)")
    sc = ax.scatter(sel[:, 0], sel[:, 1],
                    c=np.arange(len(sel)), cmap="viridis",
                    edgecolors="k", s=60, zorder=5, linewidths=0.5)
    plt.colorbar(sc, ax=ax, label="Selection order")
    ax.set_title(f"{key}\n選択順序（紫=早期, 黄=後期）", fontsize=9, fontweight="bold")
    ax.set_xlabel("x1"); ax.set_ylabel("x2")
    ax.legend(fontsize=7, loc="upper right")

    # ─ 下段：最終モデルの予測不確実性 ────────────────────────────────────────
    ax = axes[1, col]
    c = ax.contourf(X1_grid, X2_grid, Y_std, levels=20, cmap="YlOrRd")
    plt.colorbar(c, ax=ax, label="GPR std")
    ax.scatter(X_tr[:, 0], X_tr[:, 1], c="k", s=8, zorder=5)
    final_rmse = res["rmse_history"][-1]
    mean_std   = float(Y_std.mean())
    ax.set_title(f"最終 GPR 不確実性\nRMSE={final_rmse:.4f}, mean std={mean_std:.4f}",
                 fontsize=9)
    ax.set_xlabel("x1"); ax.set_ylabel("x2")

plt.tight_layout()
plt.show()

In [ ]:
# ─── カバレッジ指標：最大カバレッジ半径（k-center objective）────────────────

def coverage_radius(X_train, X_eval):
    """
    k-Center カバレッジ半径：評価グリッドの各点から学習データへの最小距離の最大値。
    値が小さいほど空間をよくカバーしている（多様性の定量指標）。
    """
    dists = cdist(X_eval, X_train, metric="euclidean")
    return float(dists.min(axis=1).max())


def mean_min_distance(X_train, X_eval):
    """評価グリッド各点から学習データへの最小距離の平均（カバレッジの平均指標）"""
    dists = cdist(X_eval, X_train, metric="euclidean")
    return float(dists.min(axis=1).mean())


print(f"\n{'':=<80}")
print(f"{'戦略':<35} {'最終RMSE':>10}  {'最大カバレッジ半径':>18}  {'平均最小距離':>12}")
print(f"{'':=<80}")

all_compare_keys = [
    "Uncertainty (GPR std)",
    "Diversity (K-Center)",
    "Hybrid α=0.7 (uncertainty-heavy)",
    "Hybrid α=0.5 (balanced)",
    "Hybrid α=0.3 (diversity-heavy)",
    "Random",
]

for key in all_compare_keys:
    if key not in results:
        continue
    res   = results[key]
    rmse  = res["rmse_history"][-1]
    rad   = coverage_radius(res["X_train_final"], X_grid_flat)
    mmd   = mean_min_distance(res["X_train_final"], X_grid_flat)
    print(f"  {key:<33}: {rmse:>10.4f}  {rad:>18.4f}  {mmd:>12.4f}")

print(f"{'':=<80}")
print("\n【指標の読み方】")
print("  最大カバレッジ半径: 最も遠い未探索点までの距離。小さいほど均等にカバー")
print("  平均最小距離:       平均的な未探索点までの距離。小さいほど全体的にカバー")

In [ ]:
# ─── カバレッジマップ：「最も遠い未探索点」の可視化 ──────────────────────────
plot_keys = [
    ("Uncertainty (GPR std)",   "tomato"),
    ("Diversity (K-Center)",    "steelblue"),
    ("Hybrid α=0.5 (balanced)", "mediumorchid"),
    ("Random",                  "gray"),
]

fig, axes = plt.subplots(1, len(plot_keys), figsize=(16, 5))
fig.suptitle("カバレッジマップ（グリッド各点 → 最近傍学習データ距離）\n"
             "赤い領域ほど未探索（遠い）", fontsize=11, fontweight="bold")

vmax_all = max(
    cdist(X_grid_flat, results[k]["X_train_final"]).min(axis=1).max()
    for k, _ in plot_keys if k in results
)

for ax, (key, color) in zip(axes, plot_keys):
    if key not in results:
        ax.set_visible(False)
        continue
    X_tr = results[key]["X_train_final"]
    dist_map = cdist(X_grid_flat, X_tr).min(axis=1).reshape(X1_grid.shape)

    c = ax.contourf(X1_grid, X2_grid, dist_map, levels=25,
                    cmap="RdYlGn_r", vmin=0, vmax=vmax_all)
    fig.colorbar(c, ax=ax, label="Min dist to training")
    ax.scatter(X_tr[:, 0], X_tr[:, 1], c="k", s=8, zorder=5)

    rad = coverage_radius(X_tr, X_grid_flat)
    ax.set_title(f"{key}\n最大カバレッジ半径 = {rad:.3f}",
                 fontsize=9, fontweight="bold", color=color)
    ax.set_xlabel("x1")

axes[0].set_ylabel("x2")
plt.tight_layout()
plt.show()

## 7. ガウス過程回帰から見た各戦略の違い

同じ GPR を最終モデルとして使ったとき、戦略ごとに「どこが得意・不得意か」を  
**予測値・不確実性・絶対誤差**の 3 つの視点から比較します。

In [ ]:
compare_3 = [
    ("Uncertainty (GPR std)",   "不確実性サンプリング",   "tomato"),
    ("Diversity (K-Center)",    "多様性サンプリング",     "steelblue"),
    ("Hybrid α=0.5 (balanced)", "ハイブリッド (α=0.5)",  "mediumorchid"),
]

fig, axes = plt.subplots(3, 3, figsize=(16, 13))
col_titles = ["GPR 予測値", "GPR 不確実性 (std)", "絶対誤差 |pred - true|"]

for row, (key, label, color) in enumerate(compare_3):
    res  = results[key]
    gpr  = res["gpr_final"]
    X_tr = res["X_train_final"]
    n_added = len(X_tr) - len(initial["X"])

    y_pred, y_std = gpr.predict(X_grid_flat, return_std=True)
    Y_pred = y_pred.reshape(X1_grid.shape)
    Y_std  = y_std.reshape(X1_grid.shape)
    Y_err  = np.abs(Y_pred - Y_grid)

    # 予測値
    ax = axes[row, 0]
    c = ax.contourf(X1_grid, X2_grid, Y_pred, levels=20, cmap="RdBu_r")
    fig.colorbar(c, ax=ax)
    ax.scatter(X_tr[:, 0], X_tr[:, 1], c="k", s=8, zorder=5)
    ax.set_ylabel(f"{label}\n(+{n_added}pts, RMSE={res['rmse_history'][-1]:.4f})",
                  fontsize=9, color=color, fontweight="bold")
    if row == 0:
        ax.set_title(col_titles[0])

    # 不確実性
    ax = axes[row, 1]
    c = ax.contourf(X1_grid, X2_grid, Y_std, levels=20, cmap="YlOrRd")
    fig.colorbar(c, ax=ax)
    ax.scatter(X_tr[:, 0], X_tr[:, 1], c="k", s=8, zorder=5)
    if row == 0:
        ax.set_title(col_titles[1])

    # 絶対誤差
    ax = axes[row, 2]
    c = ax.contourf(X1_grid, X2_grid, Y_err, levels=20, cmap="YlOrRd")
    fig.colorbar(c, ax=ax)
    ax.scatter(X_tr[:, 0], X_tr[:, 1], c="k", s=8, zorder=5)
    if row == 0:
        ax.set_title(col_titles[2])

for ax in axes[2]:
    ax.set_xlabel("x1")

fig.suptitle(f"最終 GPR モデル比較 (初期20点 + 追加{N_ITER}点)",
             fontsize=12, fontweight="bold")
plt.tight_layout()
plt.show()

## 8. まとめ：いつどの戦略を使うか

### 観察された傾向

| 戦略 | RMSE | カバレッジ | 特徴 |
|------|------|------------|------|
| **不確実性サンプリング** | 最高精度になりやすい | 偏りが生じやすい | モデルの弱点を直撃 |
| **多様性サンプリング** | 安定して改善 | 最も均等 | GPR不要で計算効率的 |
| **ハイブリッド (α=0.5)** | バランス良く改善 | 良好 | 両者の利点を統合 |
| **ランダム** | ベースライン | ランダム | 比較用 |

### 実践での選択ガイド

```
【不確実性サンプリングを選ぶとき】
  ✓ サロゲートモデルが既に適切で信頼できる
  ✓ 関数の局所的な複雑さが既知で、そこに集中したい
  ✓ GPR の再学習コストが許容できる

【多様性サンプリングを選ぶとき】
  ✓ 初期探索フェーズ（関数の全体像を知りたい）
  ✓ GPR の再学習が重い（大規模データ）
  ✓ 空間の均一なカバレッジが重要（材料探索の実験計画など）

【ハイブリッドを選ぶとき】
  ✓ 「精度」と「カバレッジ」の両立が必要
  ✓ デフォルト推奨：α=0.5 から始めて調整
  ✓ フェーズに応じて α を変える（初期: α低め → 後期: α高め）
```

### 発展的な多様性サンプリング手法

| 手法 | 概要 |
|------|------|
| **BADGE** | 勾配の多様性を最大化（分類問題向け） |
| **Core-Set** | K-Center のバリアント（厳密な近似保証あり） |
| **BALD** | Mutual Information（GPベースの情報理論的手法） |
| **Determinantal Point Process (DPP)** | カーネルの行列式で多様性を定量化 |